In [7]:
# Scoring a model on neural data can be done in a single line using the `score_physiology` method.
# This call is agnostic of a specific model implementation, it will simply look up the model name 
# in `neurality.models.models` and use the implementation defined there (also see `examples/model-activations.ipynb`).
# By default, the pre-defined layers of a model will be used to retrieve the activations, 
# but you can also pass your own.
# Just like with the model implementations, the result of this method call will be cached 
# so that it only needs to be computed once.

from neurality import score_physiology

score = score_physiology(model='alexnet', neural_data='dicarlo.Majaj2015')
# the resulting score object now contains three fields: `center`, `error` and `values`.
# `values` are the raw scores across cross-validations and possibly neurons.
# The `center` and `error` then are computed over the raw `values` 
# and yield e.g. mean and standard deviation in a unified way so that plots across models can be compared.
# In the default case, there will be one score for each combination of layers and brain regions.
print("center\n", score.center)
# In the Brain-Score paper, we reported the maximum score across layers, i.e.:
print("\nlayer max\n", score.center.max('layer'))


center
 <xarray.DataAssembly (layer: 7, region: 2)>
array([[0.349049, 0.58051 ],
       [0.451555, 0.589968],
       [0.481759, 0.581112],
       [0.496249, 0.542313],
       [0.523878, 0.482408],
       [0.480724, 0.385361],
       [0.424411, 0.278921]])
Coordinates:
  * layer    (layer) object 'features.2' 'features.5' 'features.7' ...
  * region   (region) object 'IT' 'V4'

layer max
 <xarray.DataAssembly (region: 2)>
array([0.523878, 0.589968])
Coordinates:
  * region   (region) object 'IT' 'V4'
